# LangChain Basics

In [1]:
# !pip installs the packages in the base environment
# pip - installs the packages in the virtual environment
!pip install -r ./python_requirements.txt -q

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.300
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/sm/miniconda3/envs/llm/lib/python3.11/site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
# To upgrae a library
!pip install langchain --upgrade -q

In [4]:
!pip show langchain

Name: langchain
Version: 0.0.300
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/sm/miniconda3/envs/llm/lib/python3.11/site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Python-dotenv

In [6]:
# similar to dotenv we used in vs code
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

os.environ.get('PINECONE_API_KEY')

### LLM Models (Wrappers): GPT-3

In [9]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003',temperature=0.7,max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [10]:
output = llm("explain quantum mechanics in one sentence")
print(output)



Quantum mechanics is a branch of physics that describes the behavior of matter and energy at the atomic and subatomic levels.


In [11]:
print(llm.get_num_tokens("explain quantum mechanics in one sentence"))

7


In [12]:
input_to_model = ['... is the capital of France.','What is the formula for the area of rectangle?']
output = llm.generate(input_to_model)

In [13]:
for idx, generation in enumerate(output.generations):
    print('Answer for ',input_to_model[idx],' is ', generation[0].text,' \n')

Answer for  ... is the capital of France.  is  

Paris.  

Answer for  What is the formula for the area of rectangle?  is  

The formula for the area of a rectangle is A = l x w, where A is the area, l is the length, and w is the width.  



In [11]:
output = llm.generate(['Write an original tagline for Indian Restaurant']*3) # 3 different taglines
print(output)

generations=[[Generation(text='\n\n"Experience the flavors of India - A Taste of Tradition!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Experience the Taste of India - Authentic Flavors from Across the Subcontinent"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Authentic Flavors of India, Crafted with Love!"', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 24, 'completion_tokens': 49, 'total_tokens': 73}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('74715da3-fb58-4c69-b758-cb34c22411a4')), RunInfo(run_id=UUID('2a343ac4-479a-4c5a-ba71-9f60f51e784f')), RunInfo(run_id=UUID('ca69cafd-0b22-4024-8e48-b42328435f9b'))]


In [12]:
for idx, generation in enumerate(output.generations):
    print(generation[0].text)



"Experience the flavors of India - A Taste of Tradition!"


"Experience the Taste of India - Authentic Flavors from Across the Subcontinent"


"Authentic Flavors of India, Crafted with Love!"


### ChatModels: GPT 3.5 (Turbo) & GPT 4
- Variations to the usual language models
- Rather exposing text in and text out kinda interface they expose conversation in and conversation out kinda of interface.
- Reference [Click here for documentation](https://platform.openai.com/docs/guides/gpt/chat-completions-api)

In [14]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

In [19]:
chat = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5,max_tokens=1024)
messages = [
    SystemMessage(content='You are a mathematician and respond only in English.'),
    HumanMessage(content='Explain discrete mathematics in one sentence')
]
output = chat(messages)

In [21]:
print(output.content)

Discrete mathematics is the branch of mathematics that deals with mathematical structures and objects that are fundamentally separate and distinct, rather than continuous.


### Prompt Templates
- Prompt refers to the input to the LLM models.
- PTs are a way to create dynamic prompt.

In [23]:
from langchain import PromptTemplate

In [24]:
template = '''You are an experienced Chef. 
Write a reviews about the {dish} in {language}.'''

prompt = PromptTemplate(
    input_variables=['dish','language']
    ,template=template)
print(prompt)

input_variables=['dish', 'language'] output_parser=None partial_variables={} template='You are an experienced Chef. \nWrite a reviews about the {dish} in {language}.' template_format='f-string' validate_template=True


In [27]:
from langchain.llms import OpenAI
llm = OpenAI(model='text-davinci-003',temperature=0.7)
output = llm(prompt.format(dish='Chicken Murgh Musallam',language='English'))
print(output)



Chicken Murgh Musallam is an amazing dish! The flavors of the marinade are perfectly balanced, providing a delightful combination of savory and sweet. The chicken is tender and juicy, and the spices add a delicious depth of flavor. The accompanying sauce is creamy and flavorful, adding a nice richness to the dish. Overall, I highly recommend Chicken Murgh Musallam to anyone looking for a delicious and unique dish.


### Simple Chains
- LLMs are often limited to what they can do on their own.
- For simple task you can use LLMs on their own but for complex task they are not ideal.
- Chains allow us to combine multiple components together to create a single coherent application.
- EX: We can create a chain that takes user input, formats it with a prompt template and then passed the formatted response to an LLM.

### Types of Chains
- Simple 
- Sequential
- Custom

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0.5)

template1 = '''You are an experience virologist.
Write a few sentences about the following {virus} in {language}.'''

template2 = '''You are an experienced Chef. 
Write a short review about the {dish} in English.'''

prompt1 = PromptTemplate(
    input_variables=['virus','language']
    ,template=template1)

prompt2 = PromptTemplate(
    input_variables=['dish']
    ,template=template2)

chain1 = LLMChain(llm=llm,prompt=prompt1)
chain2 = LLMChain(llm=llm,prompt=prompt2)


# If the prompt template has two variables you create 
# a dictionary(object)like this to pass to the template.
output1 = chain1.run({'virus':'HSV','language':'French'})

# But if the prompt template only has 1 variable then you just pass
# string variable like this

output2 = chain2.run('Rogan Josh');

In [31]:
print(output1)

L'HSV, ou Herpes Simplex Virus, est un virus couramment rencontré chez les êtres humains. Il existe deux types d'HSV : le HSV-1, responsable principalement des infections buccales et labiales, et le HSV-2, responsable des infections génitales. Ces virus peuvent provoquer des éruptions cutanées douloureuses et récurrentes, ainsi que des lésions sur les muqueuses. Bien qu'il n'existe pas de traitement curatif, des médicaments antiviraux peuvent aider à réduire la fréquence et la gravité des poussées d'herpès. La prévention des contacts directs avec les lésions et l'utilisation de préservatifs peuvent également aider à réduire la transmission de l'HSV.


In [34]:
print(output2)

Title: A Taste of Authenticity: Rogan Josh

As an experienced chef, I have had the privilege of indulging in a wide array of culinary delights from around the world. However, few dishes captivate my senses quite like the timeless Indian classic, Rogan Josh. This aromatic and richly flavored dish has earned its reputation as one of the crown jewels of Indian cuisine.

Originating from the beautiful region of Kashmir, Rogan Josh is a slow-cooked lamb curry that effortlessly marries a symphony of spices, creating a truly unforgettable taste experience. The dish derives its name from the Persian words "rogan" meaning oil and "josh" meaning heat, perfectly encapsulating the essence of this fiery yet balanced creation.

What sets Rogan Josh apart is its unique blend of spices, including Kashmiri red chili, ginger, fennel, and a medley of other aromatic ingredients. The result is a deep, luscious gravy that envelops tender pieces of lamb, infusing them with a harmonious blend of flavors that 

### Simple Chains Conclusion:
- We discussed simple chains which are the most basic type of chains.
- They consist of a single LLM that is used to <ins>perform a single task</ins>.
- **Ex**: Simple chain used to generate a text by passing a prompt to a LLM.

### Sequential Chains
<img src="./images/sequential-chains.png" style="border-radius:5px"></img>

Ex: Simple Sequential Chain

**Input1** --> | LLM 1 |--> **Output 1**/**Input 2** --> | LLM 2| --> **Output 2**

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain,SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003',temperature=0.7,max_tokens=1024)
prompt1= PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Java Programmer.
    Write a function that implements the concept of {concept}.''')

chain1 = LLMChain(llm=llm1, prompt=prompt1)

llm2 = OpenAI(model_name='gpt-3.5-turbo',temperature=1.2)
prompt2= PromptTemplate(
    input_variables=['function'],
    template='''Given the Java Function {function}, 
    describe it as detailed as possible.''')

chain2 = LLMChain(llm=llm2, prompt=prompt2)

# The order of specifying the chains in chains argument is very imp.
# switch them and see how the output changes.
simple_sequential_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose = True)
output = simple_sequential_chain.run('linear regression')



> Entering new SimpleSequentialChain chain...


public static double linearRegression(List<Double> x, List<Double> y) {
    // Check if the two lists are of equal lengths
    if (x.size() != y.size()) {
        throw new IllegalArgumentException("x and y must have the same number of elements");
    }

    // Calculate the mean of x and y
    double sumX = 0;
    double sumY = 0;
    for (int i = 0; i < x.size(); i++) {
        sumX += x.get(i);
        sumY += y.get(i);
    }
    double meanX = sumX / x.size();
    double meanY = sumY / y.size();

    // Calculate the slope of the linear regression line 
    double numerator = 0;
    double denominator = 0;
    for (int i = 0; i < x.size(); i++) {
        numerator += (x.get(i) - meanX) * (y.get(i) - meanY);
        denominator += (x.get(i) - meanX) * (x.get(i) - meanX);
    }
    double slope = numerator / denominator;

    // Calculate the y-intercept of the linear regression line
    double yIntercept = meanY - (slope * meanX);

 

In [37]:
output2 = simple_sequential_chain.run('softmax');



> Entering new SimpleSequentialChain chain...


public static double[] softmax(double[] values) {
    double[] output = new double[values.length];
    double sum = 0;
    for (int i = 0; i < values.length; i++) {
        output[i] = Math.exp(values[i]);
        sum += output[i];
    }
    for (int i = 0; i < values.length; i++) {
        output[i] /= sum;
    }
    return output;
}
The given Java function is a method named "softmax" that takes in an array of double values as input and returns an array of double values as the output.

First, it creates a new array called "output" with the same length as the input array, values. This array will store the results of the softmax operation for each element in the input array.

Then, it initializes a variable named "sum" to zero. This variable will be used to store the sum of the exponential values of each element in the input array.

Next, it iterates over each element in the input array using a for loop. Inside the loop, it calculates th

### LangChain Agents
- Go to chat.openai.com and ask what is the answer to 5.1 ** 7.3
- You will get wrong answer. Why ?
- Because it is a complex operation and it cannot be done by AI model.
- LLM are powerful tools but they are not perfect. 

- Task 2: Ask it anything about the most recent technology after year 2021. It will give you a made up/incorrect answer. (It was trained on data that cuts off in late 2021)

- Conclusion: There are plugins that help LLM access browser and updated information, plugins are external tools that are similar to agents. One solution is to use Agents.


### LangChain Agents in Action

In [41]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [49]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# agent_executor.run('Calculate the square root of the factorial of 20 \
# and display it with 5 decimal points.')
agent_executor.run('Calculate 5.1 ** 1.7 and display with 4 decimal points.')



> Entering new AgentExecutor chain...
 I need to use the power operator and format the output
Action: Python_REPL
Action Input: print('{:.4f}'.format(5.1 ** 1.7))
Observation: 15.9540

Thought: I now know the final answer
Final Answer: 15.9540

> Finished chain.


'15.9540'

### Embeddings
<img src="./images/embeddings.png" style="border-radius:5px" />

- The distance between 2 embeddings or 2 vectors measures their relatedness which translates to the relatedness between the text concepts they represent.
- Similar embeddings or vectors represent similar concepts.

<img src="./images/embeddings-example.png" style="border-radius:5px" />

### Intro to Vector Databases

**Challenges**
<img src="./images/llm-challenges.png"/>

- LLMs, semantic search and AI requires a large amounts of data to train & operate.
- Vector embeddings is a way of representing text as a set of numbers & numbers represent the meaning of the words in the text.
- These embeddings if you try to store in csv or a local file, the size of the file will increase dramatically & performance will drop. Hence the need for vector database. (Ex: Pinecone, Chroma, Drant etc..)


**Vector Database**
<img src="./images/vector-db.png"/>

**SQL vs. Vector Databases**
- SQL has a fixed schema while vector DB we apply similartiy methods to find a vector that is most similar to our search text.
e- Vector databases use a combination of different optimized algorithms that all participate in **<ins>Approximate Nearest Neighbor (ANN)</ins>** search.

### How Vector DB works
<img src="./images/vector-db-works.png"/>

- Vector embedding model is used for both storing and retrieving the vector from vector DB.

### Splitting & Embedding text using LangChain

[Continued over here..](./Vector-Database.ipynb)